## **Federated Learning for attack detection: 3 nodes sharing gradients**

IDs from this file = **id6xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_11172/797642493.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_11172/797642493.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_11172/797642493.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_11172/797642493.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_mem

In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id600.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [12]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id600.hdf5')

Epoch 1/15
54/54 [==============================] - 45s 826ms/step - loss: 0.2711 - accuracy: 0.9490 - val_loss: 2.6933 - val_accuracy: 0.4715
Epoch 2/15
54/54 [==============================] - 44s 817ms/step - loss: 0.0351 - accuracy: 0.9920 - val_loss: 0.9535 - val_accuracy: 0.4829
Epoch 3/15
54/54 [==============================] - 59s 1s/step - loss: 0.0263 - accuracy: 0.9923 - val_loss: 0.8317 - val_accuracy: 0.5034
Epoch 4/15
54/54 [==============================] - 67s 1s/step - loss: 0.0241 - accuracy: 0.9923 - val_loss: 0.7612 - val_accuracy: 0.5034
Epoch 5/15
54/54 [==============================] - 73s 1s/step - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.6078 - val_accuracy: 0.5825
Epoch 6/15
54/54 [==============================] - 74s 1s/step - loss: 0.0209 - accuracy: 0.9932 - val_loss: 0.5200 - val_accuracy: 0.6525
Epoch 7/15
54/54 [==============================] - 62s 1s/step - loss: 0.0193 - accuracy: 0.9937 - val_loss: 0.6459 - val_accuracy: 0.5728
Epoch 8/15
54/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 33s 19ms/step - loss: 0.7674 - accuracy: 0.7600
Epoch 1/15
54/54 [==============================] - 65s 1s/step - loss: 0.0111 - accuracy: 0.9969 - val_loss: 1.0450 - val_accuracy: 0.6792
Epoch 2/15
54/54 [==============================] - 61s 1s/step - loss: 0.0100 - accuracy: 0.9972 - val_loss: 1.2379 - val_accuracy: 0.6414
Epoch 3/15
54/54 [==============================] - 64s 1s/step - loss: 0.0103 - accuracy: 0.9972 - val_loss: 1.0654 - val_accuracy: 0.6791
Epoch 4/15
54/54 [==============================] - 60s 1s/step - loss: 0.0094 - accuracy: 0.9974 - val_loss: 0.9996 - val_accuracy: 0.6823
Epoch 5/15
54/54 [==============================] - 66s 1s/step - loss: 0.0091 - accuracy: 0.9974 - val_loss: 0.9091 - val_accuracy: 0.6864
Epoch 6/15
54/54 [==============================] - 62s 1s/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 1.1720 - val_accuracy: 0.6570
Epoch 7/15
54/54 [==============================] - 66s 1s/step - l

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 40s 23ms/step - loss: 1.9885 - accuracy: 0.6568
Epoch 1/15
54/54 [==============================] - 64s 1s/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 1.1086 - val_accuracy: 0.6766
Epoch 2/15
54/54 [==============================] - 62s 1s/step - loss: 0.0073 - accuracy: 0.9980 - val_loss: 1.0684 - val_accuracy: 0.6767
Epoch 3/15
54/54 [==============================] - 63s 1s/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 1.3797 - val_accuracy: 0.6431
Epoch 4/15
54/54 [==============================] - 61s 1s/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 1.2886 - val_accuracy: 0.6520
Epoch 5/15
54/54 [==============================] - 66s 1s/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 1.2460 - val_accuracy: 0.6559
Epoch 6/15
54/54 [==============================] - 65s 1s/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 1.5426 - val_accuracy: 0.6312
Epoch 7/15
54/54 [==============================] - 64s 1s/step - l

...

1721/1721 [==============================] - 29s 17ms/step - loss: 3.5059 - accuracy: 0.8754
Epoch 1/15
54/54 [==============================] - 57s 1s/step - loss: 0.0036 - accuracy: 0.9988 - val_loss: 1.3809 - val_accuracy: 0.6423
Epoch 2/15
54/54 [==============================] - 52s 956ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 1.6639 - val_accuracy: 0.6271
Epoch 3/15
54/54 [==============================] - 52s 968ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 1.1198 - val_accuracy: 0.6978
Epoch 4/15
54/54 [==============================] - 52s 955ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 1.6275 - val_accuracy: 0.6244
Epoch 5/15
54/54 [==============================] - 49s 905ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 0.9406 - val_accuracy: 0.7061
Epoch 6/15
54/54 [==============================] - 54s 1s/step - loss: 0.0029 - accuracy: 0.9989 - val_loss: 1.0172 - val_accuracy: 0.6922
Epoch 7/15
54/54 [==============================] - 56s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 28s 16ms/step - loss: 2.7175 - accuracy: 0.9122
Epoch 1/15
54/54 [==============================] - 45s 838ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 1.2688 - val_accuracy: 0.6874
Epoch 2/15
54/54 [==============================] - 42s 778ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 1.0658 - val_accuracy: 0.6817
Epoch 3/15
54/54 [==============================] - 36s 661ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 1.4413 - val_accuracy: 0.6327
Epoch 4/15
54/54 [==============================] - 37s 686ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 1.2432 - val_accuracy: 0.6666
Epoch 5/15
54/54 [==============================] - 38s 707ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 1.3713 - val_accuracy: 0.6657
Epoch 6/15
54/54 [==============================] - 40s 745ms/step - loss: 0.0024 - accuracy: 0.9992 - val_loss: 1.0603 - val_accuracy: 0.6985
Epoch 7/15
54/54 [==============================]

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 2.2940 - accuracy: 0.9080


In [13]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.27303168177604675, 0.767395555973053], [1.4864134788513184, 1.988478422164917], [3.059551477432251, 2.3411474227905273], [4.030683517456055, 2.9348199367523193], [4.79466438293457, 3.5389466285705566], [6.169393539428711, 3.8523378372192383], [5.884477138519287, 3.590357542037964], [5.663670063018799, 3.5059311389923096], [4.263436317443848, 2.717472791671753], [3.003235340118408, 2.2939751148223877]]
Accuracy for iterations:  [[0.8311154246330261, 0.7599542140960693], [0.5936705470085144, 0.6567790508270264], [0.5879298448562622, 0.7279481291770935], [0.5834599137306213, 0.7581195831298828], [0.5807064175605774, 0.7777555584907532], [0.5651348233222961, 0.8460001349449158], [0.5638712644577026, 0.8708857297897339], [0.5724385976791382, 0.8754450082778931], [0.5650982856750488, 0.9122102856636047], [0.5839273929595947, 0.9079960584640503]]
F1 for iterations:  [[0.8252422004921448, 0.7378583448746595], [0.5031123471935158, 0.5895755884799824], [0.50122409469637

#### Dataset 3B 

In [14]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_11172/1167559547.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_11172/1167559547.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part2.csv')


In [15]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [16]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id601.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [17]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [18]:
global_model = build_model((24,1,1))

In [19]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

KeyboardInterrupt: 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id601.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### Dataset 3C

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id602.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id602.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### Dataset 3D

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id603.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id603.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### DATASET 3E

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id604.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id604.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### DATASET 3F

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id605.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id605.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### DATASET 3G

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id606.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id606.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

### Experimenting with epochs and iterations 

Use of dataset 3C, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id610.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id610.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### #Epochs = 5, #Iterations = 5

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id611.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id611.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)